In [70]:
# Load all the pickle file
import numpy as np
import pandas as pd 
import pickle
import tensorflow
from tensorflow.keras.models import load_model

In [71]:
# Load the ANN trained model , Scaler.pkl, onehot.pkl
model = load_model('model.h5')

# load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
    
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [72]:
# Example input data
input_data = {
    "CreditScore": 700,
    "Geography": "France",
    "Gender": "Male",
    "Age": 35,
    "Tenure": 5,
    "Balance": 12000.0,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 75000.0,
}

In [73]:
# Transform using a DataFrame with the appropriate feature name
geo_encoded = onehot_encoder_geo.transform(pd.DataFrame([[input_data['Geography']]], columns=['Geography'])).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [74]:
geo_encoded

array([[1., 0., 0.]])

In [75]:
# Convert input_data dictionary to a DataFrame
input_data_df = pd.DataFrame([input_data])

# Reset the index of both DataFrames to ensure they align
input_data_df.reset_index(drop=True, inplace=True)
geo_encoded_df.reset_index(drop=True, inplace=True)

# Concatenate horizontally
input_df = pd.concat([input_data_df, geo_encoded_df], axis=1)


In [76]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,France,Male,35,5,12000.0,2,1,1,75000.0,1.0,0.0,0.0


In [77]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])


In [78]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,France,1,35,5,12000.0,2,1,1,75000.0,1.0,0.0,0.0


In [79]:
input_df.drop('Geography',axis=1,inplace=True)

In [80]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,35,5,12000.0,2,1,1,75000.0,1.0,0.0,0.0


In [81]:
# Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 0.50178794,  0.91324755, -0.37056859, -0.00134472, -1.02633868,
         0.80843615,  0.64920267,  0.97481699, -0.44216545,  1.00150113,
        -0.57946723, -0.57638802]])

In [82]:
# Predict Churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


array([[0.00273182]], dtype=float32)

In [83]:
prediction_proba = prediction[0][0]
prediction_proba

0.0027318187

In [84]:
if prediction_proba>0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [85]:
import tensorflow as tf
tf.__version__ 

'2.16.1'